In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer


In [2]:
# Load the Breast Cancer dataset
cancer = datasets.load_breast_cancer()
X, y = cancer.data, cancer.target
cancer

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

In [3]:
# Introduce missing values for demonstration
X[10, 2] = np.nan
X[30, 1] = np.nan

In [4]:
# Step 1: Handling Missing Values
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

In [5]:
X_imputed

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [6]:
# Step 3: Normalization or Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

In [7]:
# Step 4: Feature Engineering
# Introduce an artificial time column
time_column = np.arange(X_scaled.shape[0]).reshape(-1, 1)
X_with_time = np.hstack((X_scaled, time_column))

In [8]:
# Create time-based features
X_with_time = pd.DataFrame(X_with_time, columns=[f'feature_{i}' for i in range(X_scaled.shape[1])] + ['time'])
X_with_time['month'] = X_with_time['time'] % 12 + 1
X_with_time['day_of_week'] = X_with_time['time'] % 7 + 1

In [9]:
# Rolling statistics
rolling_window = 5
for i in range(X_scaled.shape[1]):
    X_with_time[f'rolling_mean_{i}'] = X_with_time[f'feature_{i}'].rolling(window=rolling_window).mean()
    X_with_time[f'rolling_std_{i}'] = X_with_time[f'feature_{i}'].rolling(window=rolling_window).std()

In [10]:
X_with_time

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,rolling_mean_25,rolling_std_25,rolling_mean_26,rolling_std_26,rolling_mean_27,rolling_std_27,rolling_mean_28,rolling_std_28,rolling_mean_29,rolling_std_29
0,1.097064,-2.074303,1.270930,0.984375,1.568466,3.283515,2.652874,2.532475,2.217515,2.255747,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.829821,-0.351816,1.687023,1.908708,-0.826962,-0.487072,-0.023846,0.548144,0.001392,-0.868652,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.579888,0.459314,1.567551,1.558884,0.942210,1.052926,1.363478,2.037231,0.939685,-0.398008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.768909,0.256531,-0.592011,-0.764464,3.283553,3.402909,1.915897,1.451707,2.867383,4.910919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.750297,-1.151293,1.777657,1.826229,0.280372,0.539340,1.371011,1.428493,-0.009560,-0.562450,...,1.369831,1.876135,1.084104,0.956643,1.648641,0.698484,1.767335,2.767868,1.391501,2.162307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,2.110995,0.725029,2.061918,2.343856,1.041842,0.219060,1.947285,2.320965,-0.312589,-0.931027,...,0.549135,1.801907,0.971266,2.229958,0.740527,1.650139,-0.520854,1.578139,0.369190,1.757761
565,1.704854,2.090898,1.616988,1.723842,0.102458,-0.017833,0.693043,1.263669,-0.217664,-1.058611,...,0.505623,1.826295,1.152516,2.096048,0.917155,1.576712,-0.416669,1.551208,0.182550,1.858814
566,0.702284,2.051274,0.673570,0.577953,-0.840484,-0.038680,0.046588,0.105777,-0.809117,-0.895587,...,0.829367,1.556893,1.479036,1.708726,1.348982,0.737801,-0.205710,1.307549,0.394792,1.688646
567,1.838341,2.342628,1.983644,1.735218,1.525767,3.272144,3.296944,2.658866,2.137194,1.043695,...,0.926569,1.760799,1.257102,1.263652,1.438514,0.833580,-0.206357,1.306233,0.207487,1.316774


In [11]:
# Additional transformations
X_with_time['log_feature_0'] = np.log1p(X_with_time['feature_0'])
X_with_time['sqrt_feature_1'] = np.sqrt(X_with_time['feature_1'])

C:\Users\Chiranjeevi\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Chiranjeevi\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [12]:
# Drop rows with missing values introduced during rolling statistics computation
X_with_time = X_with_time.dropna()

In [13]:
X_with_time

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,rolling_mean_26,rolling_std_26,rolling_mean_27,rolling_std_27,rolling_mean_28,rolling_std_28,rolling_mean_29,rolling_std_29,log_feature_0,sqrt_feature_1
6,1.170908,0.163298,1.139099e+00,1.095295,-0.123136,0.088295,0.300072,0.646935,-0.064325,-0.762332,...,1.046107,0.601904,1.392530,0.641279,1.669298,2.638618,1.393275,2.228923,0.775145,0.404101
7,-0.118517,0.361419,-7.210105e-02,-0.218965,1.604049,1.140102,0.061026,0.281950,1.403355,1.660353,...,0.870902,0.774269,1.126369,0.625442,1.534375,2.688444,1.698283,2.127067,-0.126149,0.601181
8,-0.320167,0.592171,-1.833337e-01,-0.384207,2.201839,1.684010,1.219096,1.150692,1.965600,1.572462,...,0.728991,0.550728,0.969535,0.320471,0.803138,1.285894,0.969011,1.132460,-0.385908,0.769527
9,-0.473535,1.109617,-3.287601e-01,-0.509063,1.582699,2.563358,1.738872,0.941760,0.797298,2.783096,...,1.405442,1.547701,1.147686,0.392909,1.450897,1.022093,2.417802,2.613316,-0.641570,1.053383
10,0.537556,0.923150,5.854486e-16,0.406453,-1.017686,-0.713542,-0.700684,-0.404686,-1.035476,-0.826124,...,1.031619,1.796382,0.921267,0.739984,1.115370,1.163298,1.975806,2.828552,0.430194,0.960807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,1.929226,1.354354,2.103116e+00,1.968434,0.963560,2.260135,2.870075,2.540213,1.231760,0.849484,...,0.925498,2.240153,0.359649,1.613120,-0.504029,1.567365,0.548405,1.663492,1.074738,1.163767
564,2.110995,0.725029,2.061918e+00,2.343856,1.041842,0.219060,1.947285,2.320965,-0.312589,-0.931027,...,0.971266,2.229958,0.740527,1.650139,-0.520854,1.578139,0.369190,1.757761,1.134943,0.851486
565,1.704854,2.090898,1.616988e+00,1.723842,0.102458,-0.017833,0.693043,1.263669,-0.217664,-1.058611,...,1.152516,2.096048,0.917155,1.576712,-0.416669,1.551208,0.182550,1.858814,0.995048,1.445994
566,0.702284,2.051274,6.735698e-01,0.577953,-0.840484,-0.038680,0.046588,0.105777,-0.809117,-0.895587,...,1.479036,1.708726,1.348982,0.737801,-0.205710,1.307549,0.394792,1.688646,0.531971,1.432227


In [14]:
# Drop rows with missing values introduced during rolling statistics computation
X_with_time = X_with_time.dropna()

# Update the target variable 'y' accordingly
y = y[:X_with_time.shape[0]]

# Split the Data
X_train, X_test, y_train, y_test = train_test_split(X_with_time.drop(['time'], axis=1), y, test_size=0.2, random_state=42)

In [15]:
X_train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,rolling_mean_26,rolling_std_26,rolling_mean_27,rolling_std_27,rolling_mean_28,rolling_std_28,rolling_mean_29,rolling_std_29,log_feature_0,sqrt_feature_1
302,1.693494,1.067662,1.761178,1.684024,0.828346,1.505866,1.751427,2.039810,1.596855,1.685870,...,0.127458,1.160639,0.000210,1.153491,-0.224476,0.643813,-0.039668,0.733802,0.990839,1.033277
119,1.085703,0.170290,0.916634,0.930337,-0.878202,-0.703498,-0.199239,0.181612,1.158741,-1.778754,...,0.359486,1.215894,0.190329,1.140789,0.429431,1.973769,0.395790,1.392140,0.735106,0.412663
506,-0.541698,0.177283,-0.514148,-0.573909,0.942210,0.205794,-0.088504,-0.703122,1.140487,0.870748,...,0.059545,0.405083,0.086362,0.919967,0.049899,0.278246,0.869041,1.625516,-0.780227,0.421050
413,0.245021,0.659766,0.229875,0.110382,-0.797785,-0.034889,-0.253727,-0.262045,0.483317,-0.519922,...,-0.419536,0.374416,-0.487194,0.580680,0.149231,0.638695,-0.110157,0.562801,0.219153,0.812260
503,2.545536,0.128335,2.478011,2.921209,-0.209246,0.438898,0.989340,1.325317,-1.119447,-1.128074,...,0.238876,0.671016,0.617222,0.966967,-0.042961,0.931064,0.246056,1.006229,1.265689,0.358239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,-0.825712,3.386842,-0.871740,-0.762473,-1.320851,-1.300090,-1.052512,-1.095861,0.121873,-0.640418,...,0.280403,1.235289,0.145624,1.558811,0.193558,0.393330,0.387588,1.194737,-1.747047,1.840337
26,0.128576,0.524577,0.224931,-0.028694,0.643316,1.562720,0.674211,1.003666,1.607807,0.913276,...,0.953515,0.606083,1.832579,0.448057,1.594556,1.141616,0.931328,1.043967,0.120956,0.724277
201,0.969258,0.009463,0.953711,0.843876,-0.475405,0.292971,0.185822,0.669634,-1.115796,-1.118151,...,0.253556,0.653073,0.533780,0.708151,0.455315,1.504154,-0.002539,0.753460,0.677657,0.097277
446,1.028901,2.039619,1.044345,0.929199,0.256887,0.512808,1.016961,0.877275,-0.360051,-0.515669,...,-0.053840,1.273178,-0.143103,1.156278,-0.495940,0.703159,-0.318742,0.562407,0.707494,1.428152


In [16]:
# Step 6: Pipeline with PCA and RandomForestClassifier
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=10)),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [17]:
# Fit the model
pipeline.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('pca', PCA(n_components=10)),
                ('classifier', RandomForestClassifier(random_state=42))])

In [18]:
# Step 8: Monitor Performance Metrics
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.42
